<a href="https://colab.research.google.com/github/bibhabasumohapatra/Decline-Curve-Analysis/blob/main/OIL_fields_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LIBRARIES IMPORT

In [ ]:
import pandas as pd
from scipy.optimize import curve_fit
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


---------
1. Hyperbolic


```
#  Hyperbolic decline curve equation
    Arguments:
        t: Float. Time since the well first came online, can be in various units 
        (days, months, etc) so long as they are consistent.
        qi: Float. Initial production rate when well first came online.
        b: Float. Hyperbolic decline constant
        di: Float. Nominal decline rate at time t=0
        q = qi/((1.0+b*di*t)**(1.0/b))
        return q
```


---------
2. Exponential


```
#    Exponential decline curve equation
    Arguments:
        t: Float. Time since the well first came online, can be in various units 
        (days, months, etc) so long as they are consistent.
        qi: Float. Initial production rate when well first came online.
        di: Float. Nominal decline rate (constant)
    Output: 
        Returns q
        q = qi*np.exp(-di*t)
```


---------


In [ ]:
path = '/content/drive/MyDrive/Prodn_03-2021 - Prod.csv'

In [ ]:
def hyperbolic_equation(t, qi, b, di):
    return qi/((1.0+b*di*t)**(1.0/b))

def exponential_equation(t, qi, di):
    return qi*np.exp(-di*t)

In [ ]:
def plot_actual_vs_predicted_by_equations(df, x_variable, y_variables, plot_title):
    #Plot results
    df.plot(x=x_variable, y=y_variables, title=plot_title)
    
    plt.show()

In [ ]:
df = pd.read_csv(path)

DATA PREPROCESSING

In [ ]:
wells = df.WLNO.unique()

In [ ]:
df.columns

Index(['FIELD', 'yr-no', 'MTH', 'YR', 'Month', 'FY', 'fin_yr', 'Date',
       'welno1', 'WLNO', 'SAND', 'Block', 'Lease', 'BEAN', 'WC', 'FDAYS', 'Ql',
       'LIQUID', 'WATER', 'OILCM', 'OILMT', 'Qg', 'YLIQ', 'YWAT', 'YOILCM',
       'YOILMT', 'Cumliq', 'Cumwater', 'TOILCM', 'CumoilMt'],
      dtype='object')

In [ ]:
wells

array(['MAN3', 'MAN9', 'MAN14', 'MAN10', 'MAN5', 'MAN7', 'MAN19', 'MAN23',
       'MAN27', 'MAN29', 'MAN28', 'MAN31', 'MAN33', 'MAN36', 'MAN39',
       'MAN40', 'MAN41', 'MAN42', 'MAN43', 'MAN35', 'MAN48', 'MAN22',
       'MAN25', 'MAN46', 'MAN47', 'MAN49'], dtype=object)

In [ ]:
df.head()

,FIELD,yr-no,MTH,YR,Month,FY,fin_yr,Date,welno1,WLNO,SAND,Block,Lease,BEAN,WC,FDAYS,Ql,LIQUID,WATER,OILCM,OILMT,Qg,YLIQ,YWAT,YOILCM,YOILMT,Cumliq,Cumwater,TOILCM,CumoilMt
0,MANSA,1993.33,4,1993,04/93,1993-1994,1993-94,Apr-1993,3,MAN3,NaN,NaN,#REF!,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MANSA,1993.33,4,1993,04/93,1993-1994,1993-94,Apr-1993,9,MAN9,NaN,NaN,#REF!,4.0,0.8,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MANSA,1993.42,5,1993,05/93,1993-1994,1993-94,May-1993,3,MAN3,NaN,NaN,#REF!,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MANSA,1993.42,5,1993,05/93,1993-1994,1993-94,May-1993,9,MAN9,NaN,NaN,#REF!,4.0,0.8,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MANSA,1993.50,6,1993,06/93,1993-1994,1993-94,Jun-1993,3,MAN3,NaN,NaN,#REF!,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

for well in wells:
  
  df = pd.read_csv(path)

  df = df[df['WLNO']==well]
  a = []
  for i in range(len(df)):
    a.append(i)
  temp = pd.DataFrame(a)  
  df['time'] = a
  df = df.drop(columns = ['FIELD', 'yr-no', 'MTH', 'YR', 'Month', 'FY', 'fin_yr', 'Date',
        'welno1', 'SAND', 'Block', 'Lease', 'BEAN', 'WC', 'FDAYS', 'Ql',
          'OILMT', 'Qg', 'YLIQ', 'YWAT', 'YOILCM',
        'YOILMT', 'Cumliq', 'Cumwater', 'TOILCM', 'CumoilMt'])
  df = df[(df['OILCM'].notnull()) & (df['OILCM']>0)]
  df = df[(df['LIQUID'].notnull()) & (df['LIQUID']>0)]
  df = df[(df['WATER'].notnull()) & (df['WATER']>0)]
  df_qi = df.head(5)
  qi = df_qi.OILCM.max()
  popt_exp, pcov_exp=curve_fit(exponential_equation,df['time'],df['OILCM'],bounds=(0, [qi,100]))
  popt_hyp, pcov_hyp=curve_fit(hyperbolic_equation,df['time'],df['OILCM'],bounds=(0, [qi,2,100]))

  df.loc[:,'Exponential_Predicted']=exponential_equation(df['time'], *popt_exp)
  df.loc[:,'Hyperbolic_Predicted'] = hyperbolic_equation(df['time'], *popt_hyp)

y_variables=['OILCM', "Hyperbolic_Predicted", "Exponential_Predicted"]
x_variable='time'
plot_title = well
plot_actual_vs_predicted_by_equations(df, x_variable, y_variables, plot_title)

/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


ValueError: ignored